In [0]:
from pymongo import MongoClient
import pandas as pd

In [0]:
pip install pymongo

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# connnect with mongodb
uri = "Your mongodb uri."

# Create a new client and connect to the server
client = MongoClient(uri)
db = client['inventory']
collection = db["supplychain_data"]

In [0]:
data = list(collection.find())

In [0]:
df = pd.DataFrame(data)

In [0]:
df.head(5)

,_id,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,67de468cc4c312aea5b5ff0a,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,67de468cc4c312aea5b5ff0b,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,67de468cc4c312aea5b5ff0c,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,67de468cc4c312aea5b5ff0d,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,67de468cc4c312aea5b5ff0e,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [0]:
df.drop(columns=["_id"],inplace=True)

In [0]:
df.head(5)

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col , date_format
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
spark  = SparkSession.builder.appName("supply chain inventory price prediction").getOrCreate()
inventory_df= spark.createDataFrame(df)

In [0]:
inventory_df.columns

Out[16]: ['Product type',
 'SKU',
 'Price',
 'Availability',
 'Number of products sold',
 'Revenue generated',
 'Customer demographics',
 'Stock levels',
 'Lead times',
 'Order quantities',
 'Shipping times',
 'Shipping carriers',
 'Shipping costs',
 'Supplier name',
 'Location',
 'Lead time',
 'Production volumes',
 'Manufacturing lead time',
 'Manufacturing costs',
 'Inspection results',
 'Defect rates',
 'Transportation modes',
 'Routes',
 'Costs']

In [0]:
inputColumn =["Price",'Stock levels']

In [0]:
assembler = VectorAssembler(inputCols=["Price","Stock levels"],outputCol="features")
data = assembler.transform(inventory_df).select("features","price")

In [0]:
data.show()

+--------------------+------------------+
|            features|             price|
+--------------------+------------------+
|[69.8080055421157...| 69.80800554211577|
|[14.8435232750843...| 14.84352327508434|
|[11.3196832930905...|11.319683293090566|
|[61.1633430164377...|61.163343016437736|
|[4.80549603634589...| 4.805496036345893|
|[1.69997601386593...|1.6999760138659377|
|[4.07833286310794...| 4.078332863107945|
|[42.9583843824600...| 42.95838438246007|
|[68.7175967485273...| 68.71759674852733|
|[64.0157329412785...| 64.01573294127854|
|[15.7077956819121...|15.707795681912138|
|[90.6354599822886...| 90.63545998228868|
|[71.2133890753600...| 71.21338907536008|
|[16.1603933173799...|16.160393317379977|
|[99.1713286386242...|  99.1713286386242|
|[36.9892449286269...| 36.98924492862691|
|[7.54717210979127...| 7.547172109791272|
|[81.4625343692370...| 81.46253436923702|
|[36.4436277704609...|36.443627770460935|
|[51.1238700879647...| 51.12387008796474|
+--------------------+------------

In [0]:
train_data , test_data = data.randomSplit([.8,.2])

In [0]:
rf = RandomForestRegressor(featuresCol="features",labelCol="price",numTrees=10000)
model = rf.fit(train_data)

In [0]:
prediction = model.transform(test_data)

In [0]:
evaluator = RegressionEvaluator(labelCol="price",predictionCol="prediction",metricName="rmse")
evaluator.evaluate(prediction)

Out[33]: 4.83204525726761

In [0]:
prediction.show()

+--------------------+------------------+------------------+
|            features|             price|        prediction|
+--------------------+------------------+------------------+
|[4.32434118586416...| 4.324341185864164| 10.54448527165384|
|[16.1603933173799...|16.160393317379977|17.940644764220156|
|[27.6797808865019...| 27.67978088650196|30.369071399261603|
|[42.9583843824600...| 42.95838438246007| 38.83654593588601|
|[84.8938689849508...| 84.89386898495083| 84.13890789889011|
|[8.02285921052639...| 8.022859210526393|15.484405584892126|
|[19.1274772658232...|19.127477265823256|17.940644764220156|
|[52.0759306827078...|  52.0759306827078| 60.56321540465637|
|[64.7954350001556...|  64.7954350001556| 69.41248548720813|
|[84.9577868163504...| 84.95778681635043|  83.9105091334798|
|[97.4469466178928...| 97.44694661789282| 92.64505406000946|
|[99.1132916153171...| 99.11329161531717| 93.30582157385349|
|[6.38153316274796...| 6.381533162747966|11.627768671927344|
|[17.0280279201887...|17

In [0]:
df_prediction = prediction.select("price","prediction").toPandas()

In [0]:
df_prediction

,price,prediction
0,4.324341,10.544485
1,16.160393,17.940645
2,27.679781,30.369071
3,42.958384,38.836546
4,84.893869,84.138908
5,8.022859,15.484406
6,19.127477,17.940645
7,52.075931,60.563215
8,64.795435,69.412485
9,84.957787,83.910509


In [0]:
collection_prediction  = db['prediction_data']
data_dict = df_prediction.to_dict(orient="records")
collection_prediction.insert_many(data_dict)

Out[36]: InsertManyResult([ObjectId('67deade1cbda351b787e8d33'), ObjectId('67deade1cbda351b787e8d34'), ObjectId('67deade1cbda351b787e8d35'), ObjectId('67deade1cbda351b787e8d36'), ObjectId('67deade1cbda351b787e8d37'), ObjectId('67deade1cbda351b787e8d38'), ObjectId('67deade1cbda351b787e8d39'), ObjectId('67deade1cbda351b787e8d3a'), ObjectId('67deade1cbda351b787e8d3b'), ObjectId('67deade1cbda351b787e8d3c'), ObjectId('67deade1cbda351b787e8d3d'), ObjectId('67deade1cbda351b787e8d3e'), ObjectId('67deade1cbda351b787e8d3f'), ObjectId('67deade1cbda351b787e8d40'), ObjectId('67deade1cbda351b787e8d41'), ObjectId('67deade1cbda351b787e8d42'), ObjectId('67deade1cbda351b787e8d43'), ObjectId('67deade1cbda351b787e8d44'), ObjectId('67deade1cbda351b787e8d45'), ObjectId('67deade1cbda351b787e8d46'), ObjectId('67deade1cbda351b787e8d47'), ObjectId('67deade1cbda351b787e8d48'), ObjectId('67deade1cbda351b787e8d49'), ObjectId('67deade1cbda351b787e8d4a'), ObjectId('67deade1cbda351b787e8d4b'), ObjectId('67deade1cbda3